# Query the data

## Get keys and urls

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_URL = os.getenv("OPENAI_URL")

print(f"Weaviate Key:{WEAVIATE_KEY}")
print(f"OpenAI API Key: {OPENAI_API_KEY}")
print(f"OpenAI URL: {OPENAI_URL}")



Weaviate Key:root-user-key
OpenAI API Key: sk-dummy-key-for-local-testing
OpenAI URL: http://host.docker.internal:11434


## Connect to Weaviate

In [2]:
import weaviate
from weaviate.classes.init import Auth

# Connect to the local instance
client = weaviate.connect_to_local(
  host="127.0.0.1", # the address to the learner's instance
  port=8080,
  grpc_port=50051,
  auth_credentials=Auth.api_key(WEAVIATE_KEY),
  headers={
    "X-OpenAI-Api-Key": OPENAI_API_KEY
  }
)

print(client.is_ready())

True


### Helper function

In [3]:
import json
def print_properties(item):
    print(
        json.dumps(
            item.properties,
            indent=2, sort_keys=True, default=str
        )
    )

## Vector search
[Docs - near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

In [4]:
wiki = client.collections.get("Wiki")

response = wiki.query.near_text(
    query="musical instruments",
    limit=5
)

for item in response.objects:
    print_properties(item)

{
  "text": "A chord in music is when two or more notes are played together (some text books say that it is when three or more notes are played).",
  "title": "Chord",
  "url": "https://simple.wikipedia.org/wiki/Chord",
  "wiki_id": "20231101.simple_40631_0"
}
{
  "text": "A shepherd had a lot of time to sit and think or amuse himself.  He often made simple instruments out of reeds that grew where he sat.  These instruments were often simple pan flutes or just pipes which were similar to bamboo pipes or a modern recorder.  Some shepherds learned to carve instruments out of wood.  An example is the fujara from Slovakia and southern Poland.",
  "title": "Shepherd",
  "url": "https://simple.wikipedia.org/wiki/Shepherd",
  "wiki_id": "20231101.simple_70678_5"
}
{
  "text": "Beck, John (1995). Encyclopedia of percussion instruments. New York: Garland Publishing. . Google Books preview. Accessed 8 September 2009.",
  "title": "Snare drum",
  "url": "https://simple.wikipedia.org/wiki/Snare%20

In [5]:
from weaviate.classes.query import MetadataQuery

wiki = client.collections.get("Wiki")

response = wiki.query.near_text(
    query="musical instruments",
    limit=5,
    return_metadata=MetadataQuery(distance=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.distance)

{
  "text": "A chord in music is when two or more notes are played together (some text books say that it is when three or more notes are played).",
  "title": "Chord",
  "url": "https://simple.wikipedia.org/wiki/Chord",
  "wiki_id": "20231101.simple_40631_0"
}
0.3645724058151245
{
  "text": "A shepherd had a lot of time to sit and think or amuse himself.  He often made simple instruments out of reeds that grew where he sat.  These instruments were often simple pan flutes or just pipes which were similar to bamboo pipes or a modern recorder.  Some shepherds learned to carve instruments out of wood.  An example is the fujara from Slovakia and southern Poland.",
  "title": "Shepherd",
  "url": "https://simple.wikipedia.org/wiki/Shepherd",
  "wiki_id": "20231101.simple_70678_5"
}
0.37800133228302
{
  "text": "Beck, John (1995). Encyclopedia of percussion instruments. New York: Garland Publishing. . Google Books preview. Accessed 8 September 2009.",
  "title": "Snare drum",
  "url": "https:

### Autocut

Return groups of results based on the quality/distance jumps

In [6]:
from weaviate.classes.query import MetadataQuery

wiki = client.collections.get("Wiki")

response = wiki.query.near_text(
    query="musical instruments",
    auto_limit=1,
    return_metadata=MetadataQuery(distance=True)
)

print(f"Returned object count: {len(response.objects)}")

for item in response.objects:
    print_properties(item)
    print(item.metadata.distance)

Returned object count: 1
{
  "text": "A chord in music is when two or more notes are played together (some text books say that it is when three or more notes are played).",
  "title": "Chord",
  "url": "https://simple.wikipedia.org/wiki/Chord",
  "wiki_id": "20231101.simple_40631_0"
}
0.3645724058151245


## Filters

### Fetch with filters

In [7]:
from weaviate.classes.query import Filter

wiki = client.collections.get("Wiki")

response = wiki.query.fetch_objects(
    limit=5,
    filters=Filter.by_property("title").like("music")
)

for item in response.objects:
    print_properties(item)

{
  "text": "There are different types of EDM, each varying in style, beats per minute and sound. This is a list of some of the major electronic dance music genres (each genre may additionally have further subgenres, which have not been looked at in too much detail for this section)",
  "title": "Electronic dance music",
  "url": "https://simple.wikipedia.org/wiki/Electronic%20dance%20music",
  "wiki_id": "20231101.simple_319095_3"
}
{
  "text": "In January 2013, Scottish record producer and DJ Hudson Mohawke, announced that he had signed to the production arm of G.O.O.D. Music - Very G.O.O.D. Beats. On April 2, 2013, Kid Cudi announced on Power 106 that he was no longer with the label, leaving once finding out an old vocal of his was released on Kanye's new album \"Yeezus.\" Cudi felt he was \"underused\" and would have preferred to record something particular. They left on seemingly amicable terms. On June 29, 2013 producer Hit-Boy of Very G.O.O.D. Beats announced that his contract e

In [8]:
from weaviate.classes.query import Filter

response = wiki.query.fetch_objects(
    limit=5,
    filters=Filter.by_property("title").like("m*ic") & Filter.by_property("title").not_equal("music")
)

for item in response.objects:
    print_properties(item)

{
  "text": "In 1886, Adolf Mayer first described the tobacco mosaic disease that could be transferred between plants, similar to bacterial infections.",
  "title": "Tobacco mosaic virus",
  "url": "https://simple.wikipedia.org/wiki/Tobacco%20mosaic%20virus",
  "wiki_id": "20231101.simple_528724_2"
}
{
  "text": "By its very nature, the evidence for this idea comes mainly from palaeontology. It is not claimed that this pattern is universal, but it is common. There are now a wide range of examples from many different taxa.",
  "title": "Mosaic evolution",
  "url": "https://simple.wikipedia.org/wiki/Mosaic%20evolution",
  "wiki_id": "20231101.simple_290228_4"
}
{
  "text": "\"Magic\" by Pilot is used, its 3rd use in a movie soundtrack after Disney's Herbie Fully Loaded and Happy Gilmore",
  "title": "The Magic Roundabout (movie)",
  "url": "https://simple.wikipedia.org/wiki/The%20Magic%20Roundabout%20%28movie%29",
  "wiki_id": "20231101.simple_117801_2"
}


## Search with filters
[Docs - Filters](https://weaviate.io/developers/weaviate/search/filters)

In [9]:
from weaviate.classes.query import Filter

response = wiki.query.near_text(
    query="musical instruments",
    limit=5,
    filters=Filter.by_property("title").not_equal("music")
)

for item in response.objects:
    print_properties(item)

{
  "text": "A chord in music is when two or more notes are played together (some text books say that it is when three or more notes are played).",
  "title": "Chord",
  "url": "https://simple.wikipedia.org/wiki/Chord",
  "wiki_id": "20231101.simple_40631_0"
}
{
  "text": "A shepherd had a lot of time to sit and think or amuse himself.  He often made simple instruments out of reeds that grew where he sat.  These instruments were often simple pan flutes or just pipes which were similar to bamboo pipes or a modern recorder.  Some shepherds learned to carve instruments out of wood.  An example is the fujara from Slovakia and southern Poland.",
  "title": "Shepherd",
  "url": "https://simple.wikipedia.org/wiki/Shepherd",
  "wiki_id": "20231101.simple_70678_5"
}
{
  "text": "Beck, John (1995). Encyclopedia of percussion instruments. New York: Garland Publishing. . Google Books preview. Accessed 8 September 2009.",
  "title": "Snare drum",
  "url": "https://simple.wikipedia.org/wiki/Snare%20

## Keyword Search

[Docs - keyword/bm25](https://weaviate.io/developers/weaviate/search/bm25)

In [10]:
response = wiki.query.bm25(
    query="musical instruments",
    limit=5,
)

for item in response.objects:
    print_properties(item)

{
  "text": "Equal temperament (a special case of well-temperament), in which notes of the scale which are next to one another are all separated by logarithmically equal distances, which are integer powers of 21/12.",
  "title": "Musical tuning",
  "url": "https://simple.wikipedia.org/wiki/Musical%20tuning",
  "wiki_id": "20231101.simple_72590_18"
}
{
  "text": "A shepherd had a lot of time to sit and think or amuse himself.  He often made simple instruments out of reeds that grew where he sat.  These instruments were often simple pan flutes or just pipes which were similar to bamboo pipes or a modern recorder.  Some shepherds learned to carve instruments out of wood.  An example is the fujara from Slovakia and southern Poland.",
  "title": "Shepherd",
  "url": "https://simple.wikipedia.org/wiki/Shepherd",
  "wiki_id": "20231101.simple_70678_5"
}
{
  "text": "Oklahoma is the title song from and the finale to the Broadway musical Oklahoma! It is also the state song of Oklahoma.",
  "tit

In [11]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.bm25(
    query="musical instruments",
    query_properties=["text", "title"],
    limit=5,
    return_metadata=MetadataQuery(score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)

{
  "text": "A shepherd had a lot of time to sit and think or amuse himself.  He often made simple instruments out of reeds that grew where he sat.  These instruments were often simple pan flutes or just pipes which were similar to bamboo pipes or a modern recorder.  Some shepherds learned to carve instruments out of wood.  An example is the fujara from Slovakia and southern Poland.",
  "title": "Shepherd",
  "url": "https://simple.wikipedia.org/wiki/Shepherd",
  "wiki_id": "20231101.simple_70678_5"
}
4.145349502563477
{
  "text": "Beck, John (1995). Encyclopedia of percussion instruments. New York: Garland Publishing. . Google Books preview. Accessed 8 September 2009.",
  "title": "Snare drum",
  "url": "https://simple.wikipedia.org/wiki/Snare%20drum",
  "wiki_id": "20231101.simple_306810_3"
}
3.840554714202881
{
  "text": "Oklahoma is the title song from and the finale to the Broadway musical Oklahoma! It is also the state song of Oklahoma.",
  "title": "Oklahoma! (song)",
  "url": "

In [12]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.bm25(
    query="musical instruments",
    query_properties=["text", "title^3"],
    limit=5,
    return_metadata=MetadataQuery(score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)

{
  "text": "Equal temperament (a special case of well-temperament), in which notes of the scale which are next to one another are all separated by logarithmically equal distances, which are integer powers of 21/12.",
  "title": "Musical tuning",
  "url": "https://simple.wikipedia.org/wiki/Musical%20tuning",
  "wiki_id": "20231101.simple_72590_18"
}
9.643296241760254
{
  "text": "A shepherd had a lot of time to sit and think or amuse himself.  He often made simple instruments out of reeds that grew where he sat.  These instruments were often simple pan flutes or just pipes which were similar to bamboo pipes or a modern recorder.  Some shepherds learned to carve instruments out of wood.  An example is the fujara from Slovakia and southern Poland.",
  "title": "Shepherd",
  "url": "https://simple.wikipedia.org/wiki/Shepherd",
  "wiki_id": "20231101.simple_70678_5"
}
4.145349502563477
{
  "text": "Beck, John (1995). Encyclopedia of percussion instruments. New York: Garland Publishing. . G

## Hybrid search
[Docs - hybrid](https://weaviate.io/developers/weaviate/search/hybrid)

In [13]:
response = wiki.query.hybrid(
    query="musical instruments",
    alpha=0.7,
    limit=5,
)

for item in response.objects:
    print_properties(item)

{
  "text": "Equal temperament (a special case of well-temperament), in which notes of the scale which are next to one another are all separated by logarithmically equal distances, which are integer powers of 21/12.",
  "title": "Musical tuning",
  "url": "https://simple.wikipedia.org/wiki/Musical%20tuning",
  "wiki_id": "20231101.simple_72590_18"
}
{
  "text": "A shepherd had a lot of time to sit and think or amuse himself.  He often made simple instruments out of reeds that grew where he sat.  These instruments were often simple pan flutes or just pipes which were similar to bamboo pipes or a modern recorder.  Some shepherds learned to carve instruments out of wood.  An example is the fujara from Slovakia and southern Poland.",
  "title": "Shepherd",
  "url": "https://simple.wikipedia.org/wiki/Shepherd",
  "wiki_id": "20231101.simple_70678_5"
}
{
  "text": "Beck, John (1995). Encyclopedia of percussion instruments. New York: Garland Publishing. . Google Books preview. Accessed 8 Sept

### Hybrid - select properties

In [14]:
response = wiki.query.hybrid(
    query="musical instruments",
    alpha=0.7,
    limit=5,
    query_properties=["title"]
)

for item in response.objects:
    print_properties(item)

{
  "text": "Equal temperament (a special case of well-temperament), in which notes of the scale which are next to one another are all separated by logarithmically equal distances, which are integer powers of 21/12.",
  "title": "Musical tuning",
  "url": "https://simple.wikipedia.org/wiki/Musical%20tuning",
  "wiki_id": "20231101.simple_72590_18"
}
{
  "text": "A chord in music is when two or more notes are played together (some text books say that it is when three or more notes are played).",
  "title": "Chord",
  "url": "https://simple.wikipedia.org/wiki/Chord",
  "wiki_id": "20231101.simple_40631_0"
}
{
  "text": "A shepherd had a lot of time to sit and think or amuse himself.  He often made simple instruments out of reeds that grew where he sat.  These instruments were often simple pan flutes or just pipes which were similar to bamboo pipes or a modern recorder.  Some shepherds learned to carve instruments out of wood.  An example is the fujara from Slovakia and southern Poland.",

### Hybrid - Explain score

In [15]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.hybrid(
    query="musical instruments",
    alpha=0.7,
    limit=5,
    query_properties=["title"],
    return_metadata=MetadataQuery(score=True, explain_score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)
    print(item.metadata.explain_score)

{
  "text": "Equal temperament (a special case of well-temperament), in which notes of the scale which are next to one another are all separated by logarithmically equal distances, which are integer powers of 21/12.",
  "title": "Musical tuning",
  "url": "https://simple.wikipedia.org/wiki/Musical%20tuning",
  "wiki_id": "20231101.simple_72590_18"
}
0.8661721348762512

Hybrid (Result Set keyword,bm25) Document 2488a78e-13a0-5cbe-ada0-e974525296f8: original score 3.6434345, normalized score: 0.3 - 
Hybrid (Result Set vector,hybridVector) Document 2488a78e-13a0-5cbe-ada0-e974525296f8: original score 0.6065079, normalized score: 0.5661721
{
  "text": "A chord in music is when two or more notes are played together (some text books say that it is when three or more notes are played).",
  "title": "Chord",
  "url": "https://simple.wikipedia.org/wiki/Chord",
  "wiki_id": "20231101.simple_40631_0"
}
0.699999988079071

Hybrid (Result Set vector,hybridVector) Document 0d04bf40-ff16-5669-b173-f06

## Close the client

In [16]:
client.close()